# Nurse Assignment Problem

护士排班问题：护士需要根据技能和人员限制被分配到医院轮班，模型的任务是在不同目标之间寻找平衡：

- 一方面要保证计划总成本低；
- 一方面要让分配尽可能地公平；（也就是不同护士之间工作的总时间的差距要尽可能地小）

> https://notebook.community/IBMDecisionOptimization/docplex-examples/examples/mp/jupyter/nurses_pandas


The Departments table lists all departments in the scope of the assignment.

| 数据 | 解释 | 
| :---: | :-----: |
| Departments | 表格列出工作分配范围内的所有部门 |
|Skills | 表格列出了所有技能 |
|Shifts | 表格列出了所有需要配备人员的班次。班次包含一个部门、一周中的一天以及开始和结束时间 | 
| Nurses | 表格列出了所有护士，并由其姓名标识。|
| NurseSkills  | 表提供了每个护士的技能 |
| SkillRequirements | 列出了给定部门和技能所需的最低人数|
| NurseVacations  | 列出了每个护士的休假天数 |
| NurseAssociations | 列出了希望一起工作的护士对 |
| NurseIncompatibilities | 列出了不想一起工作的护士对| 


--------------


> 目标函数、决策变量概述


设所有的护士的集合是 $N$，所有的shift的集合是 $S$. 在代码实现的时候要注意，我们是按照**一个周期**进行建模的（这个问题的周期就是1星期）。所以需要把整个时间轴从一天的24小时拉直到一周的168小时。

这样可以方便确定：“哪些shift的时间是重叠的”，从而处理一个隐含约束：**重叠的shift肯定不可能由同一个护士来做。**


我们的决策变量（最核心的）是 $X_{ns}$，0-1变量，表示护士 $n$ 是否被分配做 $s$ 工作。

同时用 $q^{+}_n$ 和 $q^{-}_n$ 表示护士 $n$ 相对全体护士工作总平均时长的差距，有 

$$\bar{Q} = \bar{q_n} + q^{+}_n + q^{-}_n$$

其中 $\bar{Q}$ 是所有护士的平均工作时间，$\bar{q_n}$ 表示护士n的总平均工作时间。这部分我们是要放到目标函数里的。

最后，显式表达的目标函数有护士的工作成本。这部分直接 $\sum \sum c_{ns} X_{ns}$。 不赘述。

---------


> 约束条件

1. 承担每个shift的护士人数不能小于规定人数，也不能大于规定人数；
2. 重叠的shift不能由同一个护士来做；
3. 护士放假的时间不能安排工作；
4. 希望一起工作的护士要安排在同样的shift中工作；
5. 不愿意一起工作的护士不能安排在一样的shift中工作；
6. 给定部门的shift必须要有满足人数的掌握一定技能的护士完成；



In [2]:
import pandas as pd
import numpy as np
from collections import namedtuple
# import seaborn as sns
import gurobipy as gp
from gurobipy import GRB

In [ ]:
departments = pd.read_excel("../../assets/data/nurses_data.xlsx", sheet_name= "Departments")
skills = pd.read_excel('../../assets/data/nurses_data.xlsx', sheet_name="Skills")
shifts = pd.read_excel("../../assets/data/nurses_data.xlsx", sheet_name= "Shifts")
skillrequirements = pd.read_excel("../../assets/data/nurses_data.xlsx", sheet_name= "SkillRequirements")
nurses = pd.read_excel("../../assets/data/nurses_data.xlsx", sheet_name= "Nurses")
nurseskills = pd.read_excel("../../assets/data/nurses_data.xlsx", sheet_name= "NurseSkills")
nursevacations = pd.read_excel("../../assets/data/nurses_data.xlsx", sheet_name= "NurseVacations")
nurseassociations = pd.read_excel("../../assets/data/nurses_data.xlsx", sheet_name= "NurseAssociations")
nurseincompatibilities = pd.read_excel("../../assets/data/nurses_data.xlsx", sheet_name= "NurseIncompatibilities")

_all_days = [
    "monday",
    "tuesday",
    "wednesday",
    "thursday",
    "friday",
    "saturday",
    "sunday"
]

def day_to_day_week(day):
    day_map = {day: d for d, day in enumerate(_all_days)}
    return day_map[day.lower()]

TWorkRules = namedtuple("TWorkRules", ["work_time_max"])
TVacation = namedtuple("TVacation", ["nurse", "day"])
TNursePair = namedtuple("TNursePair", ["firstNurse", "secondNurse"])
TSkillRequirement = namedtuple("TSkillRequirement", ["department", "skill", "required"])


class TNurse(namedtuple("TNurse1", ["name", "seniority", "qualification", "pay_rate"])):
    def __str__(self):
        return self.name
    

class TShift(namedtuple("TShift",
                        ["department", "day", "start_time", "end_time", "min_requirement", "max_requirement"])):

    def __str__(self):
        # keep first two characters in department, uppercase
        dept2 = self.department[0:4].upper()
        # keep 3 days of weekday
        dayname = self.day[0:3]
        return '{}_{}_{:02d}'.format(dept2, dayname, self.start_time).replace(" ", "_")

class ShiftActivity(object):
    @staticmethod
    def to_abstime(day_index, time_of_day):
        """ Convert a pair (day_index, time) into a number of hours since Monday 00:00

        :param day_index: The index of the day from 1 to 7 (Monday is 1).
        :param time_of_day: An integer number of hours.

        :return:
        """
        time = 24 * (day_index - 1)
        time += time_of_day
        return time

    def __init__(self, weekday, start_time_of_day, end_time_of_day):
        assert (start_time_of_day >= 0)
        assert (start_time_of_day <= 24)
        assert (end_time_of_day >= 0)
        assert (end_time_of_day <= 24)

        self._weekday = weekday
        self._start_time_of_day = start_time_of_day
        self._end_time_of_day = end_time_of_day
        # conversion to absolute time.
        start_day_index = day_to_day_week(self._weekday)
        self.start_time = self.to_abstime(start_day_index, start_time_of_day)
        end_day_index = start_day_index if end_time_of_day > start_time_of_day else start_day_index + 1
        self.end_time = self.to_abstime(end_day_index, end_time_of_day)
        assert self.end_time > self.start_time

    @property
    def duration(self):
        return self.end_time - self.start_time

    def overlaps(self, other_shift):
        if not isinstance(other_shift, ShiftActivity):

            return False
        else:
            return other_shift.end_time > self.start_time and other_shift.start_time < self.end_time


def load_data(shifts_, nurses_, skillrequirements_, nurse_skills, vacations_=None,
              nurse_associations_=None, nurse_imcompatibilities_=None, verbose=True):
    """_summary_

    Args:
        shifts_ (_type_): _description_ 所有的班
        nurses_ (_type_): _description_ 所有的护士
        skillrequirements_ (_type_): _description_ 所有的技能需求
        nurse_skills (_type_): _description_ 所有的护士的技能
        vacations_ (_type_, optional): _description_. Defaults to None. 护士的假期（不能安排工作）
        nurse_associations_ (_type_, optional): _description_. Defaults to None.
        nurse_imcompatibilities_ (_type_, optional): _description_. Defaults to None. （不能在一起工作的护士）
        verbose (bool, optional): _description_. Defaults to True.

    Returns:
        _type_: _description_
        
    """
    
    number_of_overlaps = 0
    # model.work_rules = DEFAULT_WORK_RULES
    
    shifts = [TShift(*shift_row) for _, shift_row in shifts_.iterrows()]
    nurses = [TNurse(*nurse_row) for _, nurse_row in nurses_.iterrows()]
    skill_requirements = [(skill_req["department"], skill_req["skill"], skill_req["required"]) for _, skill_req in skillrequirements_.iterrows()]
    nurse_skills = nurse_skills.groupby('nurse')['skill'].apply(list).to_dict()
    vacations = [TVacation(*vacation_row) for _, vacation_row in vacations_.iterrows()]
    nurse_associations = [TNursePair(*npr) for _, npr in nurse_associations_.iterrows()]
    nurse_incompatibilities = [TNursePair(*npr) for _, npr in nurse_imcompatibilities_.iterrows()]
    departments = set(sh.department for sh in shifts)
    shift_activities = {s: ShiftActivity(s.day, s.start_time, s.end_time) for s in shifts}
    nurses_by_id = {n.name: idx for idx, n in enumerate(nurses)}
    work_rules = TWorkRules(40)
    
    # 建立名字到人的反向映射
    
    if verbose:
        print('#nurses: {0}'.format(len(nurses)))
        print('#shifts: {0}'.format(len(shifts)))
        print('#vacations: {0}'.format(len(vacations)))
        print("#associations=%d" % len(nurse_associations))
        print("#incompatibilities=%d" % len(nurse_incompatibilities))

    return shifts, nurses, skill_requirements, nurse_skills, vacations, nurse_associations, \
        nurse_incompatibilities, departments, shift_activities, nurses_by_id, work_rules


if __name__ == "__main__":
    model = gp.Model("Nurses")
    shifts, nurses, skill_requirements, nurse_skills, vacations, nurse_associations, \
        nurse_incompatibilities, departments, shift_activities, nurses_by_id, work_rules = load_data(shifts, nurses, skillrequirements, nurseskills, nursevacations, nurseassociations, nurseincompatibilities, verbose = False)

    N = len(nurses)
    S = len(shifts)
    # Add variables
    nurse_assignment_vars = model.addVars(N, S, vtype=GRB.BINARY, name = [[f"{nurse}_{shift}" for shift in shifts] for nurse in nurses] )
    nurse_work_time_vars = model.addVars(N, vtype = GRB.CONTINUOUS, lb = 0, name = [f"{nurse}_work_time" for nurse in nurses])
    nurse_over_average_time_vars = model.addVars(N, vtype = GRB.CONTINUOUS, lb = 0, name = [f"{nurse}_over_avg_time" for nurse in nurses])
    nurse_under_average_time_vars = model.addVars(N, vtype = GRB.CONTINUOUS, lb = 0, name = [f"{nurse}_under_avg_time" for nurse in nurses])
    average_nurse_work_time = model.addVar(vtype = GRB.CONTINUOUS, lb = 0, name = "Average_Work_Time")
    max_work_time = work_rules.work_time_max
    # Add Constrs
    # print((nurse_assignment_vars.shape))
    
    model.addConstr(len(nurses) * average_nurse_work_time == gp.quicksum(nurse_work_time_vars[i] for i in range(N)), "average_Constr")

    for idx, nurse in enumerate(nurses):
        model.addConstr(nurse_work_time_vars[idx] == gp.quicksum(nurse_assignment_vars[idx, idxs] * shift_activities[s].duration for idxs, s in enumerate(shifts)), name = f"work_time_{nurse}_constr")
        model.addConstr(nurse_work_time_vars[idx] == average_nurse_work_time + nurse_over_average_time_vars[idx] - nurse_under_average_time_vars[idx], name = f"avg_work_time_{nurse}_constr")
        model.addConstr(nurse_work_time_vars[idx] <= max_work_time, f"max_time_{nurse}")
    
    
    v = 0
    for vac_nurse_id, vac_day in vacations:
        vac_n = nurses_by_id[vac_nurse_id]
        for idxs, shift in enumerate([s for s in shifts if s.day == vac_day]):
            v += 1
            model.addConstr(nurse_assignment_vars[vac_n, idxs] == 0, name = f"medium_vacations_{vac_n}_{vac_day}_{shift}")
    # print(v)
    number_of_overlaps = 0

    for i1 in range(S):
        for i2 in range(i1 + 1, S):
            s1 = shifts[i1]
            s2 = shifts[i2]
            if shift_activities[s1].overlaps(shift_activities[s2]):
                number_of_overlaps += 1
                for n in range(N):
                    model.addConstr(nurse_assignment_vars[n, i1] + nurse_assignment_vars[n, i2] <= 1, name = f"high_overlapping_{s1}_{s2}_{n}")
    
    for idx, s in enumerate(shifts):
        demand_min = s.min_requirement
        demand_max = s.max_requirement

        model.addConstr(gp.quicksum(nurse_assignment_vars[idx_n, idx] for idx_n, _ in enumerate(nurses)) >= demand_min, name = f"hight_req_min_{s}_{demand_min}")
        model.addConstr(gp.quicksum(nurse_assignment_vars[idx_n, idx] for idx_n, _ in enumerate(nurses)) <= demand_max, name = f"hight_req_max_{s}_{demand_max}")
        # model.addConstr(gp.quicksum(nurse_assignment_vars[idx_n, idx] for idx_n, _ in enumerate(nurses)) >= 1, name = f"mandatory_{s}")
    
    c = 0
    for (nurse1, nurse2) in nurse_associations:
        # 限制有绑定的护士之间必须一起shift
        if nurse1 in nurses_by_id and nurse2 in nurses_by_id:
            nurse_id1 = nurses_by_id[nurse1]
            nurse_id2 = nurses_by_id[nurse2]
            for idx, shift in enumerate(shifts):
                c += 1
                model.addConstr(nurse_assignment_vars[nurse_id1, idx] == nurse_assignment_vars[nurse_id2, idx], name = f"medium_ct_nurse_association_{nurse1}_{nurse2}_{c}")
    
    inc = 0
    for (nurse1, nurse2) in nurse_incompatibilities:
        if nurse1 in nurses_by_id and nurse2 in nurses_by_id:
            nurse_id1 = nurses_by_id[nurse1]
            nurse_id2 = nurses_by_id[nurse2]
            for idx, shift in enumerate(shifts):
                inc += 1
                model.addConstr(nurse_assignment_vars[nurse_id1, idx] + nurse_assignment_vars[nurse_id2, idx] <= 1)
    # 目标函数：总的指派数量
    
    total_number_of_assignments = gp.quicksum(nurse_assignment_vars[n, s] for n in range(N) for s in range(S))
    total_salary_cost = gp.quicksum(nurse_assignment_vars[n, idx] * nurses[n].pay_rate * shift_activities[s].duration for n in range(N) for idx, s in enumerate(shifts))
    
    total_over_average_worktime = gp.quicksum(nurse_over_average_time_vars[n] for n in range(N))
    total_under_average_worktime = gp.quicksum(nurse_under_average_time_vars[n] for n in range(N))
    
    model.setObjective(total_salary_cost + total_over_average_worktime + total_under_average_worktime, GRB.MINIMIZE)
    # model.setObjective(total_over_average_worktime + total_under_average_worktime, GRB.MINIMIZE)

    
    model.optimize()
    
    if model.status == GRB.Status.OPTIMAL:
        print(f"obj = {model.objVal} ")

    else:
        model.computeIIS()
        # model.write("model_infeasible22.ilp")
    # model.write("model_infeasible22.lp")
    print(f"Num of Binary Variables: {model.NumBinVars}")
    print(f"Num of All Variables: {model.NumVars}")
    print(f"Num of All Constrs: {model.NumConstrs}")
    
    # print(nurses_by_id)

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M3 Pro
Thread count: 11 physical cores, 11 logical processors, using up to 11 threads

Optimize a model with 1590 rows, 1409 columns and 6745 nonzeros
Model fingerprint: 0xa0cd8ce8
Variable types: 97 continuous, 1312 integer (1312 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [1e+00, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 773 rows and 305 columns
Presolve time: 0.00s
Presolved: 817 rows, 1104 columns, 4672 nonzeros
Variable types: 0 continuous, 1104 integer (1043 binary)

Root relaxation: objective 2.888200e+04, 389 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 28882.0000    0    4          - 28882.0000      -     -    0s
H    0     0  

In [ ]:

all_vars = model.getVars()
values = model.getAttr("X", all_vars)
names = model.getAttr("VarName", all_vars)
# for name, val in zip(names, values):
#     # pass
#     # print(f"{name} = {val}")
#     if val != 0:
#         print(name, val)

for j in range(S):
    cnt = 0
    for i in range(N):
        if nurse_assignment_vars[i, j].x > 0:
            cnt += 1
    print(f"Shift {j} has {cnt} people")
     
for n in range(N):
    cnt = 0
    for j in range(S):
        if nurse_assignment_vars[n, j].x > 0:
            cnt += shift_activities[shifts[j]].duration
    print(f"Nurse {nurses[n]} has {cnt} Time")
    print()
# for shift in shifts:
#     print(shift)


Shift 0 has 3 people
Shift 1 has 4 people
Shift 2 has 2 people
Shift 3 has 3 people
Shift 4 has 10 people
Shift 5 has 8 people
Shift 6 has 10 people
Shift 7 has 8 people
Shift 8 has 4 people
Shift 9 has 2 people
Shift 10 has 3 people
Shift 11 has 10 people
Shift 12 has 8 people
Shift 13 has 4 people
Shift 14 has 2 people
Shift 15 has 3 people
Shift 16 has 3 people
Shift 17 has 4 people
Shift 18 has 2 people
Shift 19 has 3 people
Shift 20 has 10 people
Shift 21 has 8 people
Shift 22 has 3 people
Shift 23 has 4 people
Shift 24 has 2 people
Shift 25 has 3 people
Shift 26 has 10 people
Shift 27 has 8 people
Shift 28 has 3 people
Shift 29 has 4 people
Shift 30 has 2 people
Shift 31 has 3 people
Shift 32 has 10 people
Shift 33 has 8 people
Shift 34 has 5 people
Shift 35 has 7 people
Shift 36 has 12 people
Shift 37 has 5 people
Shift 38 has 7 people
Shift 39 has 8 people
Shift 40 has 2 people
Nurse Anne has 40 Time

Nurse Bethanie has 40 Time

Nurse Betsy has 40 Time

Nurse Cathy has 40 Time


In [1]:
# --------------------------------------------------------------------------
# Source file provided under Apache License, Version 2.0, January 2004,
# http://www.apache.org/licenses/
# (c) Copyright IBM Corp. 2015, 2018
# --------------------------------------------------------------------------

from collections import namedtuple

from docplex.mp.model import Model
from docplex.util.environment import get_environment

# ----------------------------------------------------------------------------
# Initialize the problem data
# ----------------------------------------------------------------------------

# utility to convert a weekday string to an index in 0..6
_all_days = ["monday",
             "tuesday",
             "wednesday",
             "thursday",
             "friday",
             "saturday",
             "sunday"]


def day_to_day_week(day):
    day_map = {day: d for d, day in enumerate(_all_days)}
    return day_map[day.lower()]


TWorkRules = namedtuple("TWorkRules", ["work_time_max"])
TVacation = namedtuple("TVacation", ["nurse", "day"])
TNursePair = namedtuple("TNursePair", ["firstNurse", "secondNurse"])
TSkillRequirement = namedtuple("TSkillRequirement", ["department", "skill", "required"])


NURSES = [("Anne", 11, 1, 25),
          ("Bethanie", 4, 5, 28),
          ("Betsy", 2, 2, 17),
          ("Cathy", 2, 2, 17),
          ("Cecilia", 9, 5, 38),
          ("Chris", 11, 4, 38),
          ("Cindy", 5, 2, 21),
          ("David", 1, 2, 15),
          ("Debbie", 7, 2, 24),
          ("Dee", 3, 3, 21),
          ("Gloria", 8, 2, 25),
          ("Isabelle", 3, 1, 16),
          ("Jane", 3, 4, 23),
          ("Janelle", 4, 3, 22),
          ("Janice", 2, 2, 17),
          ("Jemma", 2, 4, 22),
          ("Joan", 5, 3, 24),
          ("Joyce", 8, 3, 29),
          ("Jude", 4, 3, 22),
          ("Julie", 6, 2, 22),
          ("Juliet", 7, 4, 31),
          ("Kate", 5, 3, 24),
          ("Nancy", 8, 4, 32),
          ("Nathalie", 9, 5, 38),
          ("Nicole", 0, 2, 14),
          ("Patricia", 1, 1, 13),
          ("Patrick", 6, 1, 19),
          ("Roberta", 3, 5, 26),
          ("Suzanne", 5, 1, 18),
          ("Vickie", 7, 1, 20),
          ("Wendie", 5, 2, 21),
          ("Zoe", 8, 3, 29)
          ]

SHIFTS = [("Emergency", "monday", 2, 8, 3, 5),
          ("Emergency", "monday", 8, 12, 4, 7),
          ("Emergency", "monday", 12, 18, 2, 5),
          ("Emergency", "monday", 18, 2, 3, 7),
          ("Consultation", "monday", 8, 12, 10, 13),
          ("Consultation", "monday", 12, 18, 8, 12),
          ("Cardiac_Care", "monday", 8, 12, 10, 13),
          ("Cardiac_Care", "monday", 12, 18, 8, 12),
          ("Emergency", "tuesday", 8, 12, 4, 7),
          ("Emergency", "tuesday", 12, 18, 2, 5),
          ("Emergency", "tuesday", 18, 2, 3, 7),
          ("Consultation", "tuesday", 8, 12, 10, 13),
          ("Consultation", "tuesday", 12, 18, 8, 12),
          ("Cardiac_Care", "tuesday", 8, 12, 4, 7),
          ("Cardiac_Care", "tuesday", 12, 18, 2, 5),
          ("Cardiac_Care", "tuesday", 18, 2, 3, 7),
          ("Emergency", "wednesday", 2, 8, 3, 5),
          ("Emergency", "wednesday", 8, 12, 4, 7),
          ("Emergency", "wednesday", 12, 18, 2, 5),
          ("Emergency", "wednesday", 18, 2, 3, 7),
          ("Consultation", "wednesday", 8, 12, 10, 13),
          ("Consultation", "wednesday", 12, 18, 8, 12),
          ("Emergency", "thursday", 2, 8, 3, 5),
          ("Emergency", "thursday", 8, 12, 4, 7),
          ("Emergency", "thursday", 12, 18, 2, 5),
          ("Emergency", "thursday", 18, 2, 3, 7),
          ("Consultation", "thursday", 8, 12, 10, 13),
          ("Consultation", "thursday", 12, 18, 8, 12),
          ("Emergency", "friday", 2, 8, 3, 5),
          ("Emergency", "friday", 8, 12, 4, 7),
          ("Emergency", "friday", 12, 18, 2, 5),
          ("Emergency", "friday", 18, 2, 3, 7),
          ("Consultation", "friday", 8, 12, 10, 13),
          ("Consultation", "friday", 12, 18, 8, 12),
          ("Emergency", "saturday", 2, 12, 5, 7),
          ("Emergency", "saturday", 12, 20, 7, 9),
          ("Emergency", "saturday", 20, 2, 12, 12),
          ("Emergency", "sunday", 2, 12, 5, 7),
          ("Emergency", "sunday", 12, 20, 7, 9),
          ("Emergency", "sunday", 20, 2, 12, 12),
          ("Geriatrics", "sunday", 8, 10, 2, 5)]

NURSE_SKILLS = {"Anne": ["Anaesthesiology", "Oncology", "Pediatrics"],
                "Betsy": ["Cardiac_Care"],
                "Cathy": ["Anaesthesiology"],
                "Cecilia": ["Anaesthesiology", "Oncology", "Pediatrics"],
                "Chris": ["Cardiac_Care", "Oncology", "Geriatrics"],
                "Gloria": ["Pediatrics"], "Jemma": ["Cardiac_Care"],
                "Joyce": ["Anaesthesiology", "Pediatrics"],
                "Julie": ["Geriatrics"], "Juliet": ["Pediatrics"],
                "Kate": ["Pediatrics"], "Nancy": ["Cardiac_Care"],
                "Nathalie": ["Anaesthesiology", "Geriatrics"],
                "Patrick": ["Oncology"], "Suzanne": ["Pediatrics"],
                "Wendie": ["Geriatrics"],
                "Zoe": ["Cardiac_Care"]
                }

VACATIONS = [("Anne", "friday"),
             ("Anne", "sunday"),
             ("Cathy", "thursday"),
             ("Cathy", "tuesday"),
             ("Joan", "thursday"),
             ("Joan", "saturday"),
             ("Juliet", "monday"),
             ("Juliet", "tuesday"),
             ("Juliet", "thursday"),
             ("Nathalie", "sunday"),
             ("Nathalie", "thursday"),
             ("Isabelle", "monday"),
             ("Isabelle", "thursday"),
             ("Patricia", "saturday"),
             ("Patricia", "wednesday"),
             ("Nicole", "friday"),
             ("Nicole", "wednesday"),
             ("Jude", "tuesday"),
             ("Jude", "friday"),
             ("Debbie", "saturday"),
             ("Debbie", "wednesday"),
             ("Joyce", "sunday"),
             ("Joyce", "thursday"),
             ("Chris", "thursday"),
             ("Chris", "tuesday"),
             ("Cecilia", "friday"),
             ("Cecilia", "wednesday"),
             ("Patrick", "saturday"),
             ("Patrick", "sunday"),
             ("Cindy", "sunday"),
             ("Dee", "tuesday"),
             ("Dee", "friday"),
             ("Jemma", "friday"),
             ("Jemma", "wednesday"),
             ("Bethanie", "wednesday"),
             ("Bethanie", "tuesday"),
             ("Betsy", "monday"),
             ("Betsy", "thursday"),
             ("David", "monday"),
             ("Gloria", "monday"),
             ("Jane", "saturday"),
             ("Jane", "sunday"),
             ("Janelle", "wednesday"),
             ("Janelle", "friday"),
             ("Julie", "sunday"),
             ("Kate", "tuesday"),
             ("Kate", "monday"),
             ("Nancy", "sunday"),
             ("Roberta", "friday"),
             ("Roberta", "saturday"),
             ("Janice", "tuesday"),
             ("Janice", "friday"),
             ("Suzanne", "monday"),
             ("Vickie", "wednesday"),
             ("Vickie", "friday"),
             ("Wendie", "thursday"),
             ("Wendie", "saturday"),
             ("Zoe", "saturday"),
             ("Zoe", "sunday")]

NURSE_ASSOCIATIONS = [("Isabelle", "Dee"),
                      ("Anne", "Patrick")]

NURSE_INCOMPATIBILITIES = [("Patricia", "Patrick"),
                           ("Janice", "Wendie"),
                           ("Suzanne", "Betsy"),
                           ("Janelle", "Jane"),
                           ("Gloria", "David"),
                           ("Dee", "Jemma"),
                           ("Bethanie", "Dee"),
                           ("Roberta", "Zoe"),
                           ("Nicole", "Patricia"),
                           ("Vickie", "Dee"),
                           ("Joan", "Anne")
                        ]

SKILL_REQUIREMENTS = [("Emergency", "Cardiac_Care", 1)]

DEFAULT_WORK_RULES = TWorkRules(40)


# ----------------------------------------------------------------------------
# Prepare the data for modeling
# ----------------------------------------------------------------------------
# subclass the namedtuple to refine the str() method as the nurse's name
class TNurse(namedtuple("TNurse1", ["name", "seniority", "qualification", "pay_rate"])):
    def __str__(self):
        return self.name


# specialized namedtuple to redefine its str() method
class TShift(namedtuple("TShift",
                        ["department", "day", "start_time", "end_time", "min_requirement", "max_requirement"])):

    def __str__(self):
        # keep first two characters in department, uppercase
        dept2 = self.department[0:4].upper()
        # keep 3 days of weekday
        dayname = self.day[0:3]
        return '{}_{}_{:02d}'.format(dept2, dayname, self.start_time).replace(" ", "_")


class ShiftActivity(object):
    @staticmethod
    def to_abstime(day_index, time_of_day):
        """ Convert a pair (day_index, time) into a number of hours since Monday 00:00

        :param day_index: The index of the day from 1 to 7 (Monday is 1).
        :param time_of_day: An integer number of hours.

        :return:
        """
        time = 24 * (day_index - 1)
        time += time_of_day
        return time

    def __init__(self, weekday, start_time_of_day, end_time_of_day):
        assert (start_time_of_day >= 0)
        assert (start_time_of_day <= 24)
        assert (end_time_of_day >= 0)
        assert (end_time_of_day <= 24)

        self._weekday = weekday
        self._start_time_of_day = start_time_of_day
        self._end_time_of_day = end_time_of_day
        # conversion to absolute time.
        start_day_index = day_to_day_week(self._weekday)
        self.start_time = self.to_abstime(start_day_index, start_time_of_day)
        end_day_index = start_day_index if end_time_of_day > start_time_of_day else start_day_index + 1
        self.end_time = self.to_abstime(end_day_index, end_time_of_day)
        assert self.end_time > self.start_time

    @property
    def duration(self):
        return self.end_time - self.start_time

    def overlaps(self, other_shift):
        if not isinstance(other_shift, ShiftActivity):
            return False
        else:
            return other_shift.end_time > self.start_time and other_shift.start_time < self.end_time


def solve(model, **kwargs):
    # Here, we set the number of threads for CPLEX to 2 and set the time limit to 2mins.
    model.parameters.threads = 2
    model.parameters.timelimit = 120  # nurse should not take more than that !
    sol = model.solve(log_output=True, **kwargs)
    if sol is not None:
        print("solution for a cost of {}".format(model.objective_value))
        print_information(model)
        print_solution(model)
        return model.objective_value
    else:
        print("* model is infeasible")
        return None


def load_data(model, shifts_, nurses_, nurse_skills, vacations_=None,
              nurse_associations_=None, nurse_imcompatibilities_=None, verbose=True):
    """ Usage: load_data(shifts, nurses, nurse_skills, vacations) """
    model.number_of_overlaps = 0
    model.work_rules = DEFAULT_WORK_RULES
    model.shifts = [TShift(*shift_row) for shift_row in shifts_]
    model.nurses = [TNurse(*nurse_row) for nurse_row in nurses_]
    model.skill_requirements = SKILL_REQUIREMENTS
    model.nurse_skills = nurse_skills
    # transactional data
    model.vacations = [TVacation(*vacation_row) for vacation_row in vacations_] if vacations_ else []
    model.nurse_associations = [TNursePair(*npr) for npr in nurse_associations_]\
    if nurse_associations_ else []
    model.nurse_incompatibilities = [TNursePair(*npr) for npr in nurse_imcompatibilities_]\
    if nurse_imcompatibilities_ else []

    # computed
    model.departments = set(sh.department for sh in model.shifts)

    if verbose:
        print('#nurses: {0}'.format(len(model.nurses)))
        print('#shifts: {0}'.format(len(model.shifts)))
        print('#vacations: {0}'.format(len(model.vacations)))
        print("#associations=%d" % len(model.nurse_associations))
        print("#incompatibilities=%d" % len(model.nurse_incompatibilities))


def setup_data(model):
    """ compute internal data """
    # compute shift activities (start, end duration) and stor ethem in a dict indexed by shifts
    model.shift_activities = {s: ShiftActivity(s.day, s.start_time, s.end_time) for s in model.shifts}
    # map from nurse names to nurse tuples.
    model.nurses_by_id = {n.name: n for n in model.nurses}


def setup_variables(model):
    all_nurses, all_shifts = model.nurses, model.shifts
    # one binary variable for each pair (nurse, shift) equal to 1 iff nurse n is assigned to shift s
    model.nurse_assignment_vars = model.binary_var_matrix(all_nurses, all_shifts, 'NurseAssigned')
    # for each nurse, allocate one variable for work time
    model.nurse_work_time_vars = model.continuous_var_dict(all_nurses, lb=0, name='NurseWorkTime')
    # and two variables for over_average and under-average work time
    model.nurse_over_average_time_vars = model.continuous_var_dict(all_nurses, lb=0,
                                                                   name='NurseOverAverageWorkTime')
    model.nurse_under_average_time_vars = model.continuous_var_dict(all_nurses, lb=0,
                                                                    name='NurseUnderAverageWorkTime')
    # finally the global average work time
    model.average_nurse_work_time = model.continuous_var(lb=0, name='AverageWorkTime')


def setup_constraints(model):
    all_nurses = model.nurses
    all_shifts = model.shifts
    nurse_assigned = model.nurse_assignment_vars
    nurse_work_time = model.nurse_work_time_vars
    shift_activities = model.shift_activities
    nurses_by_id = model.nurses_by_id
    max_work_time = model.work_rules.work_time_max

    # define average
    model.add_constraint(
        len(all_nurses) * model.average_nurse_work_time == model.sum(nurse_work_time[n] for n in all_nurses), "average")

    # compute nurse work time , average and under, over
    for n in all_nurses:
        work_time_var = nurse_work_time[n]
        model.add_constraint(
            work_time_var == model.sum(nurse_assigned[n, s] * shift_activities[s].duration for s in all_shifts),
            "work_time_{0!s}".format(n))

        # relate over/under average worktime variables to the worktime variables
        # the trick here is that variables have zero lower bound
        # however, thse variables are not completely defined by this constraint,
        # only their difference is.
        # if these variables are part of the objective, CPLEX wil naturally minimize their value,
        # as expected
        model.add_constraint(
            work_time_var == model.average_nurse_work_time
            + model.nurse_over_average_time_vars[n]
            - model.nurse_under_average_time_vars[n],
            "average_work_time_{0!s}".format(n))

        # state the maximum work time as a constraint, so that is can be relaxed,
        # should the problem become infeasible.
        model.add_constraint(work_time_var <= max_work_time, "max_time_{0!s}".format(n))

    # vacations
    v = 0
    for vac_nurse_id, vac_day in model.vacations:
        vac_n = nurses_by_id[vac_nurse_id]
        for shift in (s for s in all_shifts if s.day == vac_day):
            v += 1
            model.add_constraint(nurse_assigned[vac_n, shift] == 0,
                                 "medium_vacations_{0!s}_{1!s}_{2!s}".format(vac_n, vac_day, shift))
    #print('#vacation cts: {0}'.format(v))

    # a nurse cannot be assigned overlapping shifts
    # post only one constraint per couple(s1, s2)
    number_of_overlaps = 0
    nb_shifts = len(all_shifts)
    for i1 in range(nb_shifts):
        for i2 in range(i1 + 1, nb_shifts):
            s1 = all_shifts[i1]
            s2 = all_shifts[i2]
            if shift_activities[s1].overlaps(shift_activities[s2]):
                number_of_overlaps += 1
                for n in all_nurses:
                    model.add_constraint(nurse_assigned[n, s1] + nurse_assigned[n, s2] <= 1,
                                         "high_overlapping_{0!s}_{1!s}_{2!s}".format(s1, s2, n))
    #print('# overlapping cts: {0}'.format(number_of_overlaps))

    for s in all_shifts:
        demand_min = s.min_requirement
        demand_max = s.max_requirement
        total_assigned = model.sum(nurse_assigned[n, s] for n in model.nurses)
        model.add_constraint(total_assigned >= demand_min,
                             "high_req_min_{0!s}_{1}".format(s, demand_min))
        model.add_constraint(total_assigned <= demand_max,
                             "medium_req_max_{0!s}_{1}".format(s, demand_max))
        model.add_constraint(total_assigned >= 1, "mandatory_presence_{0!s}".format(s))

    for (dept, skill, required) in model.skill_requirements:
        if required > 0:
            for dsh in (s for s in all_shifts if dept == s.department):
                model.add_constraint(model.sum(nurse_assigned[skilled_nurse, dsh] for skilled_nurse in
                                               (n for n in all_nurses if
                                                n.name in model.nurse_skills.keys() and skill in model.nurse_skills[
                                                    n.name])) >= required,
                                     "high_required_{0!s}_{1!s}_{2!s}_{3!s}".format(dept, skill, required, dsh))

    # nurse-nurse associations
    # for each pair of associated nurses, their assignment variables are equal
    # over all shifts.
    c = 0
    for (nurse_id1, nurse_id2) in model.nurse_associations:
        if nurse_id1 in nurses_by_id and nurse_id2 in nurses_by_id:
            nurse1 = nurses_by_id[nurse_id1]
            nurse2 = nurses_by_id[nurse_id2]
            for s in all_shifts:
                c += 1
                ctname = 'medium_ct_nurse_assoc_{0!s}_{1!s}_{2:d}'.format(nurse_id1, nurse_id2, c)
                model.add_constraint(nurse_assigned[nurse1, s] == nurse_assigned[nurse2, s], ctname)

    # nurse-nurse incompatibilities
    # for each pair of incompatible nurses, the sum of assigned variables is less than one
    # in other terms, both nurses can never be assigned to the same shift
    c = 0
    for (nurse_id1, nurse_id2) in model.nurse_incompatibilities:
        if nurse_id1 in nurses_by_id and nurse_id2 in nurses_by_id:
            nurse1 = nurses_by_id[nurse_id1]
            nurse2 = nurses_by_id[nurse_id2]
            for s in all_shifts:
                c += 1
                ctname = 'medium_ct_nurse_incompat_{0!s}_{1!s}_{2:d}'.format(nurse_id1, nurse_id2, c)
                model.add_constraint(nurse_assigned[nurse1, s] + nurse_assigned[nurse2, s] <= 1, ctname)

    model.total_number_of_assignments = model.sum(nurse_assigned[n, s] for n in all_nurses for s in all_shifts)
    # model.nurse_costs = [model.nurse_assignment_vars[n, s] * n.pay_rate * model.shift_activities[s].duration
    #                      for n in model.nurses for s in model.shifts]

    def assignment_cost_f(ns):
        n, s = ns
        return n.pay_rate * model.shift_activities[s].duration

    model.nurse_costs = model.scal_prod_f(nurse_assigned, assignment_cost_f)
    model.total_salary_cost = model.sum(model.nurse_costs)


def setup_objective(model):
    model.add_kpi(model.total_salary_cost, "Total salary cost")
    model.add_kpi(model.total_number_of_assignments, "Total number of assignments")
    model.add_kpi(model.average_nurse_work_time, "average work time")

    total_over_average_worktime = model.sum(model.nurse_over_average_time_vars[n] for n in model.nurses)
    total_under_average_worktime = model.sum(model.nurse_under_average_time_vars[n] for n in model.nurses)
    model.add_kpi(total_over_average_worktime, "Total over-average worktime")
    model.add_kpi(total_under_average_worktime, "Total under-average worktime")
    total_fairness = total_over_average_worktime + total_under_average_worktime
    model.add_kpi(total_fairness, "Total fairness")

    model.minimize(model.total_salary_cost + total_fairness + model.total_number_of_assignments)


def print_information(model):
    print("#shifts=%d" % len(model.shifts))
    print("#nurses=%d" % len(model.nurses))
    print("#vacations=%d" % len(model.vacations))
    print("#nurse skills=%d" % len(model.nurse_skills))
    print("#nurse associations=%d" % len(model.nurse_associations))
    print("#incompatibilities=%d" % len(model.nurse_incompatibilities))
    model.print_information()
    model.report_kpis()


def print_solution(model):
    print("*************************** Solution ***************************")
    print("Allocation By Department:")
    for d in model.departments:
        print("\t{}: {}".format(d, sum(
            model.nurse_assignment_vars[n, s].solution_value for n in model.nurses for s in model.shifts if
            s.department == d)))
    print("Cost By Department:")
    for d in model.departments:
        cost = sum(
            model.nurse_assignment_vars[n, s].solution_value * n.pay_rate * model.shift_activities[s].duration for n in
            model.nurses for s in model.shifts if s.department == d)
        print("\t{}: {}".format(d, cost))
    print("Nurses Assignments")
    for n in sorted(model.nurses):
        total_hours = sum(
            model.nurse_assignment_vars[n, s].solution_value * model.shift_activities[s].duration for s in model.shifts)
        print("\t{}: total hours:{}".format(n.name, total_hours))
        for s in model.shifts:
            if model.nurse_assignment_vars[n, s].solution_value == 1:
                print("\t\t{}: {} {}-{}".format(s.day, s.department, s.start_time, s.end_time))


# ----------------------------------------------------------------------------
# Build the model
# ----------------------------------------------------------------------------

def build(context=None, verbose=False, **kwargs):
    mdl = Model("Nurses", context=context, **kwargs)
    load_data(mdl, SHIFTS, NURSES, NURSE_SKILLS, VACATIONS, NURSE_ASSOCIATIONS,
              NURSE_INCOMPATIBILITIES, verbose=verbose)
    setup_data(mdl)
    setup_variables(mdl)
    setup_constraints(mdl)
    setup_objective(mdl)
    return mdl


# ----------------------------------------------------------------------------
# Solve the model and display the result
# ----------------------------------------------------------------------------

if __name__ == '__main__':
    # Build model
    model = build()

    # Solve the model and print solution
    solve(model)

    # Save the CPLEX solution as "solution.json" program output
    with get_environment().get_output_stream("solution.json") as fp:
        model.solution.export(fp, "json")
    model.end()

Checking license ...
excevp: No such file or directory
execvp(/Applications/CPLEX_Studio_Community2211/cplex/bin/x86-64_osx/cpxchecklic): 2

Process died with signal 31
No license found. [0.01 s]
CPLEX Error  1016: Community Edition. Problem size limits exceeded. Purchase at http://ibm.biz/error1016.


DOcplexLimitsExceeded: **** Promotional version. Problem size limits (1000 vars, 1000 consts) exceeded, model has 1409 vars, 1760 consts, CPLEX code=1016